In [1]:
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import QtWidgets
from PyQt5 import QtGui
from PyQt5 import QtCore, QtGui, QtWidgets
from collections import deque
import threading
import os
import sys
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import copy
import cv2
import imutils
import time
from collections import Counter
counter = 0
buffer = 32
pts = deque(maxlen=buffer)
D = []
(dX,dY) = (0,0)
direction = ""
center ="None"
frame = None
# Disable tensorflow compilation warnings
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
lh = 0
ls = 48
lv = 80

hh = 20
hs = 255
hv = 255

play = True
left = True

def mask(hsv):
    global lh,ls,lv,hh,hs,hv
    lower= np.array([lh,ls,lv], dtype="uint8")
    upper = np.array([hh,hs,hv], dtype="uint8")
    # initialize the list of tracked point, the frame counter.the cordinates
    
    mask = cv2.inRange(hsv, lower,upper)
    
    mask = cv2.erode(mask, None, iterations = 2)
    mask = cv2.dilate(mask,None,iterations = 2)
    return mask
    
def Track (hsv, frame):
    
    global lh,ls,lv,hh,hs,hv
    lower= np.array([lh,ls,lv], dtype="uint8")
    upper = np.array([hh,hs,hv], dtype="uint8")
    # initialize the list of tracked point, the frame counter.the cordinates
    
    frame1 = []
    mask = cv2.inRange(hsv, lower,upper)
    mask = cv2.erode(mask, None, iterations = 2)
    mask = cv2.dilate(mask,None,iterations = 2)
    
    
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = "None"
    
    if len(cnts) > 0:
            
            c= max(cnts, key=cv2.contourArea)
            ((x,y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            
            
            if radius > 10:
                cv2.circle(frame, (int(x), int(y)), int(radius), (0,255,255),2 )
                cv2.circle(frame, center, 5, (0,0,255), -1)
                x1 = center[0] - 150
                y1 = center[1] - 200
                
                x2 = center[0] + 150
                y2 = center[1] + 100
                
                if (x1<0):
                    x1 = 0
                if (x1>320):
                    x1 = 320
                if (x2<0):
                    x2 = 0
                if (x2>320):
                    x2 = 320
                    
                if (y1<0):
                    y1 = 0
                if (y1>480):
                    y1 = 480
                if (y2<0):
                    y2 = 0
                if (y2>480):
                    y2 = 480
                global left
                if(not left):
                    x1 += 320
                    x2 += 320
                cv2.rectangle(frame,(x1,y1), (x2,y2), (255,0,0),1 )
                frame1 = frame[y1:y2, x1:x2]
                pts.appendleft(center)
                
            
            
            for i in np.arange(1,len(pts)):
                if pts[i - 1] is None or pts[i] is None:
                    
                    continue
                   
                global counter
                global direction
                global dX,dY,D
                
                if counter >= 10 and i == 10  and pts[i-10] is not None :
                    
                    
                    dX = pts[i-10][0] - pts [i][0]
                    dY = pts[i-10][1] - pts [i][1]
                    (dirX, dirY) = ("","")
                    
    
                    
                    if np.abs(dX)> 20 :
                        
                        dirX= "East" if np.sign(dX) == 1 else "West"
                        D.append(dirX)
                    if np.abs(dY) > 20 :
                        dirY= "North" if np.sign(dY) == 1 else "South"
                        D.append(dirY)
                    if dirX !="" and dirY !="":
                        direction = "{}-{}".format(dirY,dirX)
                        D.append(direction)
                    else:
                        direction  = dirX if dirX !="" else dirY
                        D.append(direction)
                
                thicknees = 2
                counter +=1  
                cv2.line(frame, pts[i-1], pts[i], (0,0,255), thicknees)
    
                  
    return frame1
                
                
                 
            
def predict(image_data):

    predictions = sess.run(softmax_tensor, \
             {'DecodeJpeg/contents:0': image_data})

    # Sort to show labels of first prediction in order of confidence
    top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]

    max_score = 0.0
    res = ''
    for node_id in top_k:
        human_string = label_lines[node_id]
        score = predictions[0][node_id]
        if score > max_score:
            max_score = score
            res = human_string
    return res, max_score

# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line
                   in tf.gfile.GFile("logs/trained_labels.txt")]

# Unpersists graph from file
with tf.gfile.FastGFile("logs/trained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

    
x = None
text = ""
norec = False
with tf.Session() as sess:  
    
    def getVowel(x,direction):
        amharic_letters = [["ሀ","ሁ","ሂ","ሃ","ሄ","ህ","ሆ"],["ለ","ሉ","ሊ","ላ","ሌ","ል","ሎ"],["ሐ","ሑ","ሒ","ሓ","ሔ","ሕ","ሖ"],["መ","ሙ","ሚ","ማ","ሜ","ም","ሞ"]]
        if (direction=="East"):
            return amharic_letters[x-1][1]
        elif(direction=="West"):
            return amharic_letters[x-1][2]
        elif(direction=="North"):
            return amharic_letters[x-1][3]
        elif(direction=="South"):
            return amharic_letters[x-1][4]
        elif(direction=="South-West" or direction=="South-East"):
            return amharic_letters[x-1][5]
        elif(direction=="North-West" or direction=="North-East"):
            return amharic_letters[x-1][6]
        elif(direction==""):
            return amharic_letters[x-1][0]
        else:
            return ""
        
    def getLetter(x):
        amharic_letters = ["ሀ","ለ","ሐ","መ"]
        return amharic_letters[x-1]

    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    
    class Thread(QThread):
        changePixmap = pyqtSignal(QImage)
        
        def run(self):
            # Feed the image_data as input to the graph and get first prediction

            c = 0

            cap = cv2.VideoCapture(0)

            res, score = '', 0.0
            i = 0
            mem = ''
            consecutive = 0
            sequence = ''
            
            
            while True:
                
                ret, frame = cap.read()
                global frame
                frame=cv2.flip(frame,1)
                global left
                
                if (left):
                    x1,y1,x2,y2 = 0,0,320,480
                else:
                    x1,y1,x2,y2 = 320,0,640,480
                
                padding = 0
                #cv2.rectangle(frame, (x1+padding, y1+ padding), (x2+padding, y2+padding), (255,0,0), 5)
                hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                hsv = hsv[y1:y2, x1:x2]
                image_cropped = Track(hsv, frame)
                #image_cropped= frame[y1:y2, x1:x2]
                global D
                global norec
                
                c += 1
                
                #cv2.rectangle(frame,(x1,y1), (x2,y2), (255,0,0),2 )
                rgbImage = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                convertToQtFormat = QImage(rgbImage.data, rgbImage.shape[1], rgbImage.shape[0], QImage.Format_RGB888)
                p = convertToQtFormat.scaled(640, 480, Qt.KeepAspectRatio)
                self.changePixmap.emit(p)
                global play
                if (not play):
                    continue
                if (len(image_cropped)==0):
                    continue
                image_data = cv2.imencode('.jpg', image_cropped)[1].tostring()
                #tensor
                
                
                #print(score)
                
                
                if i == 4:
                    res_tmp, score = predict(image_data)
                    res = res_tmp
                    i = 0
                    if mem == res:
                        consecutive += 1
                    else:
                        consecutive = 0
                    if consecutive == 2 and res not in ['nothing']:
                        global text
                    if (res.upper()=="NOTHING"):
                        text+=""
                        norec = False
                    elif (norec):
                        continue
                    elif (res.upper()=="SPACE"):
                        text+=" "
                    elif (res.upper()=="DEL"):
                        text= text[0:-2]
                    else:
                        for j in range(0,20):
                            (grabbed,img) = cap.read()
                            img=cv2.flip(img,1)
                            img = imutils.resize(img, width=600)
                            if (left):
                                x1,y1,x2,y2 = 0,0,320,480
                            else:
                                x1,y1,x2,y2 = 320,0,640,480

                            frame= img[y1:y2, x1:x2]
                            blurred = cv2.GaussianBlur(frame,(11,11),0)
                            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                            Track(hsv,frame)
                            cv2.putText(img, direction,(10,30), cv2.FONT_HERSHEY_SIMPLEX,0.65,(0,0,255),3)
                            cv2.putText(img, "dX: {},dY: {}".format(dX,dY), (10, frame.shape[0]-10),cv2.FONT_HERSHEY_SIMPLEX,0.35,(0,255,255),1)
                            rgbImage = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                            convertToQtFormat = QImage(rgbImage.data, rgbImage.shape[1], rgbImage.shape[0], QImage.Format_RGB888)
                            p = convertToQtFormat.scaled(640, 480, Qt.KeepAspectRatio)
                            self.changePixmap.emit(p)


                        norec = True

                        if D == []:
                            text += getLetter(int(res.upper()))

                        else:
                            value  = Counter(D).most_common(1)[0][0]
                            print(value)
                            text += getVowel(int(res.upper()), value)

                    D =[]
                    global x
                    pts.clear()
                    x.setText(text)
                    
                i+=1
                
                
                
     
    
               
    
    def cvmask():
        time.sleep(5)
        global frame
        while True:
            
            hsvi = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            mask1 = mask(hsvi)
            cv2.imshow("mask", mask1)
            k=cv2.waitKey(30) & 0xff
            if k == 27:
                break
        cv2.destroyAllWindows()
    class App(QWidget):
        def __init__(self):
            super().__init__()
            self.title = 'Eth SL'
            self.left = 0
            self.top = 100
            self.width = 640
            self.height = 480

            self.initUI()

        @pyqtSlot(QImage)
        def setImage(self, image):
            self.label.setPixmap(QPixmap.fromImage(image))
        def changeValuelh(self, value):
            global lh
            lh = value * 2.55
        def changeValuels(self, value):
            global ls
            ls = value * 2.55
        def changeValuelv(self, value):
            global lv
            lv = value * 2.55
        ####################################################    
        def changeValuehh(self, value):
            global hh
            hh = value * 2.55
        def changeValuehs(self, value):
            global hs
            hs = value * 2.55
        def changeValuehv(self, value):
            global hv
            hv = value * 2.55
            
        def play(self, pressed):
            global play
            self.play_label.setText("Translating...")
            play = True
        def pause(self, pressed):
            global play
            play = False
            self.play_label.setText("Paused")
            
        def leftfunc(self, pressed):
            global left
            self.left_label.setText("Left")
            left = True
        def right(self, pressed):
            global left
            left = False
            self.left_label.setText("Right")
            
        def initUI(self):
            self.setWindowTitle(self.title)
            self.setWindowIcon(QtGui.QIcon('logo.png'))
            self.setGeometry(self.left, self.top, self.width, self.height)
            self.resize(860, 550)
            
            #hsv controls
            self.hsv_title = QLabel(self)
            font = QtGui.QFont("Roboto", 16, QtGui.QFont.Bold)
            self.hsv_title.setText("HSV Adjuster")
            self.hsv_title.setFont(font)
            self.hsv_title.move(650,5)
            
            self.hsv_lower = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.hsv_lower.setText("Lower HSV")
            self.hsv_lower.setFont(font)
            self.hsv_lower.move(650,50)
            
            self.hsv_lh = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.hsv_lh.setText("H")
            self.hsv_lh.setFont(font)
            self.hsv_lh.move(650,85)
            
            lower_h = QSlider(Qt.Horizontal, self)
            lower_h.setFocusPolicy(Qt.NoFocus)
            lower_h.setGeometry(670, 80, 150, 30)
            lower_h.setValue(0)
            lower_h.valueChanged[int].connect(self.changeValuelh)
            
            self.hsv_ls = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.hsv_ls.setText("S")
            self.hsv_ls.setFont(font)
            self.hsv_ls.move(650,105)
            
            lower_s = QSlider(Qt.Horizontal, self)
            lower_s.setFocusPolicy(Qt.NoFocus)
            lower_s.setGeometry(670, 100, 150, 30)
            lower_s.setValue(48/2.55)
            lower_s.valueChanged[int].connect(self.changeValuels)
            
            self.hsv_lv = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.hsv_lv.setText("V")
            self.hsv_lv.setFont(font)
            self.hsv_lv.move(650,125)
            
            lower_v = QSlider(Qt.Horizontal, self)
            lower_v.setFocusPolicy(Qt.NoFocus)
            lower_v.setGeometry(670, 120, 150, 30)
            lower_v.setValue(80/2.55)
            lower_v.valueChanged[int].connect(self.changeValuelv)
            
            self.hsv_higher = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.hsv_higher.setText("Higher HSV")
            self.hsv_higher.setFont(font)
            self.hsv_higher.move(650,150)
            
            self.hsv_hh = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.hsv_hh.setText("H")
            self.hsv_hh.setFont(font)
            self.hsv_hh.move(650,185)
            
            higher_h = QSlider(Qt.Horizontal, self)
            higher_h.setFocusPolicy(Qt.NoFocus)
            higher_h.setGeometry(670, 180, 150, 30)
            higher_h.setValue(20/2.55)
            higher_h.valueChanged[int].connect(self.changeValuehh)
            
            self.hsv_hs = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.hsv_hs.setText("S")
            self.hsv_hs.setFont(font)
            self.hsv_hs.move(650,205)
            
            higher_s = QSlider(Qt.Horizontal, self)
            higher_s.setFocusPolicy(Qt.NoFocus)
            higher_s.setGeometry(670, 200, 150, 30)
            higher_s.setValue(255/2.55)
            higher_s.valueChanged[int].connect(self.changeValuehs)
            
            self.hsv_hv = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.hsv_hv.setText("V")
            self.hsv_hv.setFont(font)
            self.hsv_hv.move(650,225)
            
            higher_v = QSlider(Qt.Horizontal, self)
            higher_v.setFocusPolicy(Qt.NoFocus)
            higher_v.setGeometry(670, 220, 150, 30)
            higher_v.setValue(255/2.55)
            higher_v.valueChanged[int].connect(self.changeValuehv)
            
            #beba buttons
            self.play_btn = QPushButton(self)
            self.play_btn.setIcon(QtGui.QIcon('play.png'))
            self.play_btn.setIconSize(QtCore.QSize(50,50))
            self.play_btn.resize(100,100)
            self.play_btn.move(650,250)
            self.play_btn.clicked[bool].connect(self.play)
            
            self.pause_btn = QPushButton(self)
            self.pause_btn.setIcon(QtGui.QIcon('pause.png'))
            self.pause_btn.setIconSize(QtCore.QSize(50,50))
            self.pause_btn.resize(100,100)
            self.pause_btn.move(750,250)
            self.pause_btn.clicked[bool].connect(self.pause)
            
            self.left_btn = QPushButton(self)
            self.left_btn.setIcon(QtGui.QIcon('left.png'))
            self.left_btn.setIconSize(QtCore.QSize(50,50))
            self.left_btn.resize(100,100)
            self.left_btn.move(650,350)
            self.left_btn.clicked.connect(self.leftfunc)
            
            self.right_btn = QPushButton(self)
            self.right_btn.setIcon(QtGui.QIcon('right.png'))
            self.right_btn.setIconSize(QtCore.QSize(50,50))
            self.right_btn.resize(100,100)
            self.right_btn.move(750,350)
            self.right_btn.clicked[bool].connect(self.right)
            
            global play
            self.play_label = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.play_label.setText("Translating...")
            self.play_label.setFont(font)
            self.play_label.move(650,470)
            self.play_label.resize(100,30)
            
            global left
            self.left_label = QLabel(self)
            font = QtGui.QFont("Roboto", 12, QtGui.QFont.Normal)
            self.left_label.setText("Left")
            self.left_label.setFont(font)
            self.left_label.move(650,490)
            self.left_label.resize(100,30)
            
            # create a label for qt
            self.label = QLabel(self)
            self.label.move(0, 0)
            self.label.resize(640, 480)
            self.le = QLineEdit(self)
            self.le.move(0, 480)
        
            #font = self.le.font()      # lineedit current font
            #font.setPointSize(32)  # change it's size
            palette = QtGui.QPalette()
            palette.setColor(QtGui.QPalette.Text, QtCore.Qt.red)
            self.le.setPalette(palette)
            font = QtGui.QFont("Noto Sans Ethiopic", 32, QtGui.QFont.Bold)
            self.le.setFont(font)      # set font
            self.le.resize(640,70)
            global x
            x = self.le
            #self.le.setText(getVowel(1,"east"))
            self.show()
            th = Thread(self)
            
            th.changePixmap.connect(self.setImage)
            th.start()
            
            
    
               
            
    if __name__ == '__main__':
        app = QtWidgets.QApplication(sys.argv)
        app.setStyle('Fusion')
        palette = QtGui.QPalette()
        palette.setColor(QtGui.QPalette.Window, QtGui.QColor(53,53,53))
        palette.setColor(QtGui.QPalette.WindowText, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.Base, QtGui.QColor(15,15,15))
        palette.setColor(QtGui.QPalette.AlternateBase, QtGui.QColor(53,53,53))
        palette.setColor(QtGui.QPalette.ToolTipBase, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.ToolTipText, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.Text, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.Button, QtGui.QColor(53,53,53))
        palette.setColor(QtGui.QPalette.ButtonText, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.BrightText, QtCore.Qt.red)

        palette.setColor(QtGui.QPalette.Highlight, QtGui.QColor(0,116,255).lighter())
        palette.setColor(QtGui.QPalette.HighlightedText, QtCore.Qt.black)
        app.setPalette(palette)
        ex = App()
        th2 = threading.Thread(target=cvmask, args =())
        th2.start()
        exit(app.exec())

<ipython-input-1-7adef242e3ab>:234: SyntaxWarning: name 'frame' is assigned to before global declaration
  global frame


West

East



South


East

South



### 